# 設置環境並載入套件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 載入事先寫好的 functions
from compute_parameters import *

# 讀入語料

In [4]:
# 讀入 ASBC txt 檔
asbc_path = '../CorpusforDDL/ASBC_去XML標記'
all_files = os.listdir(asbc_path)

asbc_corpus = []
for f in all_files:
  with open(f'{asbc_path}/{f}') as f:
      lines = f.readlines()
      asbc_corpus.append(lines)

In [5]:
# 前處理
def preprocess_asbc(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\n', '', clean_string) # 移除換行符號
  clean_string = re.sub(r'\-+', '', clean_string) # 移除連續的 -
  clean_string = re.sub(r'\[\+[A-z0-9]+\]', '', clean_string) # 移除特徵標記
  clean_string = re.sub(r'\(\w+CATEGORY\)', '', clean_string) # 移除標點符號標記

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？',
                                                       ':': '：',
                                                       ';': '；'})) # 部分符號改回全形
  return clean_string

In [15]:
asbc_corpus_preprocessed = []

for text in asbc_corpus:
  preprocessed = [preprocess_asbc(string) for string in text]
  joined = ''.join(preprocessed)
  split_1 = re.split(r'(?<=。」|！」|？」)', joined) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平
  split = [x for x in split if len(x)>1]

  asbc_corpus_preprocessed.append(split)

## 語料格式準備 - 1
斷詞 + pos tag

In [21]:
asbc_ws_pos = asbc_corpus_preprocessed
asbc_ws_pos[0][0]

'碰到(VC) 這(Nep) 種(Nf) 情形(Na) ，真(D) 是(SHI) 會(D) 令(VL) 人(Na) 又(Caa) 氣憤(VH) 又(Caa) 苦惱(VH) 。'

## 語料格式準備 - 2
斷好詞，無標點符號

In [22]:
def asbc_get_sentence(sentence):

  sentence_words = re.sub(r'\([A-z0-9]+\)', '', sentence)
  sentence_words = re.sub(r'[^\w\s]', '', sentence_words).strip().split(' ')
  sentence_words = list(filter(None, sentence_words))

  return sentence_words

In [24]:
asbc_ws = []

for text in asbc_ws_pos:
  ws = [asbc_get_sentence(sent) for sent in text]
  asbc_ws.append(ws)

In [27]:
asbc_ws[0][0]

['碰到', '這', '種', '情形', '真', '是', '會', '令', '人', '又', '氣憤', '又', '苦惱']

## 語料格式準備 - 3
完整句子，含標點符號

In [29]:
asbc_text = []

for text in asbc_ws_pos:
  t = []
  for sent in text:
    res = re.sub(r'\([A-z0-9_]+\)', '', sent)
    res = res.replace(' ', '')
    t.append(res)
  asbc_text.append(t)

In [31]:
asbc_text[0][0]

'碰到這種情形，真是會令人又氣憤又苦惱。'

# 句子長度

In [34]:
sentence_length = []

for text in asbc_ws:
  length = [len(sentence) for sentence in text]
  sentence_length.append(length)

In [37]:
sentence_length[0][0]

13

# 高頻詞與低頻詞

In [38]:
high_low_freq = []

for text in asbc_ws:
  freq = [get_high_low_freq(sentence) for sentence in text]
  high_low_freq.append(freq)

In [39]:
high_low_freq[0][0]

{'High': 13}

# 詞頻

In [41]:
word_freq = []

for text in asbc_ws:
  freq = [get_word_freq(sentence) for sentence in text]
  word_freq.append(freq)

In [42]:
word_freq[0][0]

'碰到(15360) 這(2114032) 種(701563) 情形(35199) 真(137497) 是(4642407) 會(1014977) 令(128742) 人(1550512) 又(535555) 氣憤(2937) 又(535555) 苦惱(3892)'

# 詞彙等級

In [43]:
word_level = []

for text in asbc_ws:
  level = [get_word_level(sentence) for sentence in text]
  word_level.append(level)

In [44]:
word_level[0][0]

{'第4級': 1, '第1級': 3, 'Unknown': 1, '第3*級': 1, '第1*級': 4, '第5級': 2, '第6級': 1}

# 詞彙長度

In [45]:
long_word_count = []

for text in asbc_ws:
  count = [get_long_word_count(sentence) for sentence in text]
  long_word_count.append(count)

In [46]:
long_word_count[0][:10]

[0, 2, 1, 1, 0, 1, 1, 1, 0, 0]

# 完整的句子

In [49]:
is_complete_sentence = []

for text in asbc_ws_pos:
  sent = [get_complete_sentence(sentence) for sentence in text]
  is_complete_sentence.append(sent)

In [50]:
is_complete_sentence[0][:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

# 完整的語境

In [51]:
is_complete_context = []

for text in asbc_ws_pos:
  sent = [get_complete_context(sentence) for sentence in text]
  is_complete_context.append(sent)

In [52]:
is_complete_context[0][:10]

['Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'N']

# Black list

In [53]:
is_blacklist = []

for text in asbc_text:
  bl = [get_blacklist(x) for x in text]
  is_blacklist.append(bl)

In [54]:
is_blacklist[0][:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Grey list

In [55]:
is_greylist = []

for text in asbc_ws_pos:
  gl = [get_greylist(x) for x in text]
  is_greylist.append(gl)

In [56]:
is_greylist[0][:10] 

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Make dataframe

In [57]:
def make_asbc_df(asbc_text, asbc_ws, sentence_length, word_freq, high_low_freq, 
                 word_level, long_word_count, is_complete_sentence, is_complete_context, 
                 is_blacklist, is_greylist):
  
  asbc_ws = [' '.join(sent) for sent in asbc_ws]
  asbc_df = pd.DataFrame({'sentence': asbc_text,
                        'sentence_preprocessed': asbc_ws,
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})
  
  return asbc_df

In [66]:
asbc_dfs = []

for text, ws, length, w_freq, hl_freq, level, count, complete_sent, complet_cont, bl, gl in zip(asbc_text, asbc_ws, sentence_length, word_freq, high_low_freq, 
                 word_level, long_word_count, is_complete_sentence, is_complete_context, 
                 is_blacklist, is_greylist):
  df = make_asbc_df(text, ws, length, w_freq, hl_freq, level, count, complete_sent, complet_cont, bl, gl)
  asbc_dfs.append(df)

In [72]:
asbc_dfs = [df[df['sentence_length']>0] for df in asbc_dfs]
asbc_dfs[0]

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,碰到這種情形，真是會令人又氣憤又苦惱。,碰到 這 種 情形 真 是 會 令 人 又 氣憤 又 苦惱,13,碰到(15360) 這(2114032) 種(701563) 情形(35199) 真(137...,{'High': 13},"{'第4級': 1, '第1級': 3, 'Unknown': 1, '第3*級': 1, ...",0,Y,Y,N,N
1,相信大多數的母親也都有過類似的困擾，因為我們的生活環境逐漸被色情電影、錄影帶、畫片、書刊所污染。,相信 大多數 的 母親 也 都 有 過 類似 的 困擾 因為 我們 的 生活 環境 逐漸 被...,25,相信(82200) 大多數(21212) 的(16029838) 母親(84326) 也(1...,{'High': 25},"{'第2級': 3, 'Unknown': 3, '第1級': 7, '第3級': 3, '...",2,Y,Y,N,N
2,尤其在「笑貧不笑娼」的社會風氣裡，有些唯利是圖的不法商人，更是以誇張的色情表演和刺激的畫面來...,尤其 在 笑 貧 不 笑 娼 的 社會 風氣 裡 有 些 唯利是圖 的 不法 商人 更 是 ...,31,尤其(50700) 在(3284238) 笑(68493) 貧(3950) 不(228252...,{'High': 31},"{'第3*級': 3, '第1*級': 2, '第2級': 2, '第5級': 5, '第1...",1,Y,Y,N,N
3,要我們這些做父母的完全禁止孩子們去接觸，可真是防不勝防。,要 我們 這些 做 父母 的 完全 禁止 孩子 們 去 接觸 可 真 是 防不勝防,16,要(851264) 我們(979260) 這些(315113) 做(411649) 父母(7...,{'High': 16},"{'第1級': 8, 'Unknown': 2, '第3級': 1, '第3*級': 1, ...",1,Y,N,N,N
4,縱使你的孩子現在沒看過，以後他同樣會有機會接觸，所以，我們要做的並不是彌補的工作，而是如何「...,縱使 你 的 孩子 現在 沒 看 過 以後 他 同樣 會 有 機會 接觸 所以 我們 要 做...,31,縱使(3754) 你(1414837) 的(16029838) 孩子(242092) 現在(...,{'High': 31},"{'第6級': 3, '第1級': 15, 'Unknown': 3, '第1*級': 4,...",0,Y,Y,N,N
...,...,...,...,...,...,...,...,...,...,...,...
3065,活在世界上，誰跟我是最親密的關係？,活 在 世界 上 誰 跟 我 是 最 親密 的 關係,12,活(49727) 在(3284238) 世界(165235) 上(1233218) 誰(11...,{'High': 12},"{'第4級': 1, '第1*級': 3, '第2級': 1, '第1級': 5, '第4*...",0,Y,N,N,N
3066,我如何去肯定、維持、發展這種關係？,我 如何 去 肯定 維持 發展 這 種 關係,9,我(3033032) 如何(148068) 去(479993) 肯定(34303) 維持(3...,{'High': 9},"{'第1級': 3, '第5級': 2, '第4*級': 1, '第4級': 1, 'Unk...",0,Y,N,N,N
3067,這一點在談「自我定位」時，再詳細說明。,這 一 點 在 談 自我 定位 時 再 詳細 說明,11,這(2114032) 一(3701681) 點(118735) 在(3284238) 談(4...,{'High': 11},"{'第1級': 3, '第1*級': 2, '第3*級': 2, '第4級': 2, '第6...",0,Y,N,N,N
3068,存在先於本質，是不是意味好的存在先於好的本質？,存在 先於 本質 是 不 是 意味 好 的 存在 先於 好 的 本質,14,存在(75197) 先於(576) 本質(19801) 是(4642407) 不(22825...,{'High': 14},"{'第5級': 2, 'Unknown': 4, '第6級': 3, '第1級': 5}",0,Y,N,N,N


In [75]:
for idx, df in enumerate(asbc_dfs):
  df.to_csv(f'../results/asbc/asbc_parameters/asbc_parameters_{idx+1}.csv', index = False)

## Concordance

In [76]:
from nltk.text import Text

In [95]:
def make_concordance_df(target_words, asbc_ws):

  corpus = [item for sublist in asbc_ws for item in sublist]
  text = Text(corpus)
  dfs = []

  for word in target_words:

    con_list = text.concordance_list(word)
    right_word = [x.right[0] for x in con_list]
    left_word = [x.left[-1] for x in con_list]
    context = [x.left + [word] + x.right for x in con_list]
    context = [' '.join(x) for x in context]

    df = pd.DataFrame({'left_word': left_word,
                      'target_word': word,
                      'right_word': right_word,
                      'context': context})
    dfs.append(df)

  return dfs

In [108]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']

asbc_concordance_df = []
for text in asbc_ws:
  dfs = make_concordance_df(target_words, text)
  concordance_df = pd.concat(dfs)
  asbc_concordance_df.append(concordance_df)

In [109]:
for idx, df in enumerate(asbc_concordance_df):
  df.to_csv(f'../results/asbc/asbc_concordance_df/asbc_concordance_df_{idx+1}.csv', index = False)